# Librerias y dependecias

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from ytmusicapi import YTMusic
import re
from tqdm import tqdm
import time
import random
import requests
import os
from dotenv import load_dotenv

# Autenticación

In [3]:
load_dotenv() # carga .env en process env
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET
    )
)
yt = YTMusic()

# Filtrado de resultados TOP 100

In [4]:
#Carga del dataset
df_final = pd.read_csv("../data/interim/03_dataset_con_audiofeatures_reccobeats.csv", encoding="utf-8", low_memory=False)


In [5]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2564 entries, 0 to 2563
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artista           2564 non-null   object 
 1   tema              2564 non-null   object 
 2   genero            2564 non-null   object 
 3   popularidad       2157 non-null   float64
 4   fuente            2564 non-null   object 
 5   spotify_id        2454 non-null   object 
 6   año               2157 non-null   float64
 7   id                865 non-null    object 
 8   href              865 non-null    object 
 9   acousticness      864 non-null    float64
 10  danceability      864 non-null    float64
 11  energy            864 non-null    float64
 12  instrumentalness  864 non-null    float64
 13  key               864 non-null    float64
 14  liveness          864 non-null    float64
 15  loudness          864 non-null    float64
 16  mode              864 non-null    float64


In [6]:
print(df_final["genero"].value_counts())

genero
rock           958
hiphop         277
jazz           263
hyperpop       199
kpop           194
pop            185
regueton       148
clasica         98
electronica     96
blues           76
country         70
Name: count, dtype: int64


In [7]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2564 entries, 0 to 2563
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artista           2564 non-null   object 
 1   tema              2564 non-null   object 
 2   genero            2564 non-null   object 
 3   popularidad       2157 non-null   float64
 4   fuente            2564 non-null   object 
 5   spotify_id        2454 non-null   object 
 6   año               2157 non-null   float64
 7   id                865 non-null    object 
 8   href              865 non-null    object 
 9   acousticness      864 non-null    float64
 10  danceability      864 non-null    float64
 11  energy            864 non-null    float64
 12  instrumentalness  864 non-null    float64
 13  key               864 non-null    float64
 14  liveness          864 non-null    float64
 15  loudness          864 non-null    float64
 16  mode              864 non-null    float64


In [8]:
# Eliminar registros con nulos en acousticness
df_filtrado = df_final.dropna(subset=["acousticness"]).copy()

# Asegurar que popularidad sea numérica
df_filtrado["popularidad"] = pd.to_numeric(df_filtrado["popularidad"], errors="coerce")

# Rellenar popularidad faltante con promedio por género
df_filtrado["popularidad"] = df_filtrado.groupby("genero")["popularidad"].transform(
    lambda x: x.fillna(x.mean())
)

# Función auxiliar: top N por género
def top_por_genero(df, genero, n=100):
    subset = df[df["genero"].str.lower() == genero.lower()]
    subset_top = subset.sort_values("popularidad", ascending=False).head(n)
    return subset_top

# Definir los géneros a limitar
generos_limitados = ["rock", "kpop", "regueton","hiphop","pop"]

# Filtrar el resto (que no se tocan)
otros_generos = df_filtrado[~df_filtrado["genero"].str.lower().isin(generos_limitados)]

# Tomar top 100 de los géneros seleccionados
top_rock = top_por_genero(df_filtrado, "rock", 150)
top_kpop = top_por_genero(df_filtrado, "kpop", 70)
top_regueton = top_por_genero(df_filtrado, "regueton", 100)
top_hiphop = top_por_genero(df_filtrado, "hiphop", 150)
top_pop = top_por_genero(df_filtrado, "pop", 50)

# Unir todo: los tops + los géneros restantes sin cambios
df_final_filtrado = pd.concat([otros_generos, top_rock, top_kpop, top_regueton,top_hiphop,top_pop], ignore_index=True)

# Resumen
print(df_final_filtrado["genero"].value_counts())
print(f"✅ Dataset final: {df_final_filtrado.shape[0]} filas")

# Guardar
df_final_filtrado.to_csv("../data/processed/04_dataset_filtrado.csv", index=False)


genero
rock           150
hiphop         150
regueton        90
kpop            70
electronica     61
country         56
pop             50
hyperpop        43
jazz            39
blues           18
clasica         16
Name: count, dtype: int64
✅ Dataset final: 743 filas
